In [1]:
# !pip install vina

In [2]:
# download the molecules for a simple python_scripting example
# !git clone https://github.com/ccsb-scripps/AutoDock-Vina.git

Cloning into 'AutoDock-Vina'...
remote: Enumerating objects: 2377, done.
remote: Counting objects: 100% (183/183), done.
remote: Compressing objects: 100% (98/98), done.
remote: Total 2377 (delta 81), reused 156 (delta 71), pack-reused 2194
Receiving objects: 100% (2377/2377), 47.16 MiB | 13.49 MiB/s, done.
Resolving deltas: 100% (1416/1416), done.


In [3]:
# copy the example molecules to the current directory
# !ls ./AutoDock-Vina/example/python_scripting/*
# !cp ./AutoDock-Vina/example/python_scripting/1iep* .
!ls .

1iep_ligand.pdbqt  1iep_receptor.pdbqt	AutoDock-Vina  Untitled.ipynb


In [4]:
# %%bash
# replicate ligand into several files
# rm -f 1iep_ligand_*
# number_of_copies=3
# for i in $(seq 1 $number_of_copies); do cp 1iep_ligand.pdbqt "1iep_ligand_$i.pdbqt"; done
# ls

1iep_ligand_1.pdbqt
1iep_ligand_2.pdbqt
1iep_ligand_3.pdbqt
1iep_ligand.pdbqt
1iep_receptor.pdbqt
AutoDock-Vina
Untitled.ipynb


In [9]:
# set docking parameters
receptor_molecule = '2itx_clean.pdbqt'
vina_map_center = [-57.735, -3.936, -24.746]
vina_map_box_size = [30,30,31]
docking_exhaustiveness = 10 #32
docking_n_poses = 5 #20

# read files in directory according to pattern
# this could be changed if we know the exact name of the ligand files
import glob
ligand_molecules = sorted(glob.glob('test_*.pdbqt'))
ligand_molecules

['test_Acetamidoeugenol.pdbqt', 'test_ranitidine.pdbqt']

In [ ]:
%%time
from vina import Vina

import time

v = Vina(sf_name='vina')

v.set_receptor(receptor_molecule)

# set a time flag for each ligand
global_init_time = time.time()

for i, ligand_molecule in enumerate(ligand_molecules):
  # logging
  print("working with ligand %d out of %d" % (i, len(ligand_molecules)))
  print("name of ligand file is %s" % ligand_molecule)
  ligand_init_time = time.time()

  v.set_ligand_from_file(ligand_molecule)
  v.compute_vina_maps(center=vina_map_center, box_size=vina_map_box_size)
    
  print(v.info())

  # Score the current pose
  energy = v.score()
  print('Score before minimization: %.3f (kcal/mol)' % energy[0])

  # Minimized locally the current pose
  energy_minimized = v.optimize()
  print('Score after minimization : %.3f (kcal/mol)' % energy_minimized[0])
  v.write_pose(ligand_molecule.split('.')[0] + '_minimized.pdbqt', overwrite=True)

  # Dock the ligand
  v.dock(exhaustiveness=docking_exhaustiveness, n_poses=docking_n_poses)
  v.write_poses(ligand_molecule.split('.')[0] + '_vina_out.pdbqt', n_poses=5, overwrite=True)

  # log time in human readable format
  print("time elapsed for ligand is %s" % time.strftime("%H:%M:%S", time.gmtime(time.time() - ligand_init_time)))
  print("global time elapsed is %s" % time.strftime("%H:%M:%S", time.gmtime(time.time() - global_init_time)))
  print("-"*50)